In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
#import xgboost as xgb
from sklearn.metrics import roc_auc_score

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
file_folder = '/Users/rosleeb/ieee-fraud-detection'
train_identity = pd.read_csv(f'{file_folder}/train_identity.csv')
train_transaction = pd.read_csv(f'{file_folder}/train_transaction.csv')
test_identity = pd.read_csv(f'{file_folder}/test_identity.csv')
test_transaction = pd.read_csv(f'{file_folder}/test_transaction.csv')

In [3]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [4]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [5]:
train.rename(columns = {'id_1':'id-1'}, inplace = True)
train.rename(columns = {'id_2':'id-2'}, inplace = True)
train.rename(columns = {'id_3':'id-3'}, inplace = True)
train.rename(columns = {'id_4':'id-4'}, inplace = True)
train.rename(columns = {'id_5':'id-5'}, inplace = True)
train.rename(columns = {'id_6':'id-6'}, inplace = True)
train.rename(columns = {'id_7':'id-7'}, inplace = True)
train.rename(columns = {'id_8':'id-8'}, inplace = True)
train.rename(columns = {'id_9':'id-9'}, inplace = True)
train.rename(columns = {'id_10':'id-10'}, inplace = True)
train.rename(columns = {'id_11':'id-11'}, inplace = True)
train.rename(columns = {'id_12':'id-12'}, inplace = True)
train.rename(columns = {'id_13':'id-13'}, inplace = True)
train.rename(columns = {'id_14':'id-14'}, inplace = True)
train.rename(columns = {'id_15':'id-15'}, inplace = True)
train.rename(columns = {'id_16':'id-16'}, inplace = True)
train.rename(columns = {'id_17':'id-17'}, inplace = True)
train.rename(columns = {'id_18':'id-18'}, inplace = True)
train.rename(columns = {'id_19':'id-19'}, inplace = True)
train.rename(columns = {'id_20':'id-20'}, inplace = True)
train.rename(columns = {'id_21':'id-21'}, inplace = True)
train.rename(columns = {'id_22':'id-22'}, inplace = True)
train.rename(columns = {'id_23':'id-23'}, inplace = True)
train.rename(columns = {'id_24':'id-24'}, inplace = True)
train.rename(columns = {'id_25':'id-25'}, inplace = True)
train.rename(columns = {'id_26':'id-26'}, inplace = True)
train.rename(columns = {'id_27':'id-27'}, inplace = True)
train.rename(columns = {'id_28':'id-28'}, inplace = True)
train.rename(columns = {'id_29':'id-29'}, inplace = True)
train.rename(columns = {'id_30':'id-30'}, inplace = True)
train.rename(columns = {'id_31':'id-31'}, inplace = True)
train.rename(columns = {'id_32':'id-32'}, inplace = True)
train.rename(columns = {'id_33':'id-33'}, inplace = True)
train.rename(columns = {'id_34':'id-34'}, inplace = True)
train.rename(columns = {'id_35':'id-35'}, inplace = True)
train.rename(columns = {'id_36':'id-36'}, inplace = True)
train.rename(columns = {'id_37':'id-37'}, inplace = True)
train.rename(columns = {'id_38':'id-38'}, inplace = True)

#train.head()

In [22]:
test.rename(columns = {'id_1':'id-1'}, inplace = True)
test.rename(columns = {'id_2':'id-2'}, inplace = True)
test.rename(columns = {'id_3':'id-3'}, inplace = True)
test.rename(columns = {'id_4':'id-4'}, inplace = True)
test.rename(columns = {'id_5':'id-5'}, inplace = True)
test.rename(columns = {'id_6':'id-6'}, inplace = True)
test.rename(columns = {'id_7':'id-7'}, inplace = True)
test.rename(columns = {'id_8':'id-8'}, inplace = True)
test.rename(columns = {'id_9':'id-9'}, inplace = True)
test.rename(columns = {'id_10':'id-10'}, inplace = True)
test.rename(columns = {'id_11':'id-11'}, inplace = True)
test.rename(columns = {'id_12':'id-12'}, inplace = True)
test.rename(columns = {'id_13':'id-13'}, inplace = True)
test.rename(columns = {'id_14':'id-14'}, inplace = True)
test.rename(columns = {'id_15':'id-15'}, inplace = True)
test.rename(columns = {'id_16':'id-16'}, inplace = True)
test.rename(columns = {'id_17':'id-17'}, inplace = True)
test.rename(columns = {'id_18':'id-18'}, inplace = True)
test.rename(columns = {'id_19':'id-19'}, inplace = True)
test.rename(columns = {'id_20':'id-20'}, inplace = True)
test.rename(columns = {'id_21':'id-21'}, inplace = True)
test.rename(columns = {'id_22':'id-22'}, inplace = True)
test.rename(columns = {'id_23':'id-23'}, inplace = True)
test.rename(columns = {'id_24':'id-24'}, inplace = True)
test.rename(columns = {'id_25':'id-25'}, inplace = True)
test.rename(columns = {'id_26':'id-26'}, inplace = True)
test.rename(columns = {'id_27':'id-27'}, inplace = True)
test.rename(columns = {'id_28':'id-28'}, inplace = True)
test.rename(columns = {'id_29':'id-29'}, inplace = True)
test.rename(columns = {'id_30':'id-30'}, inplace = True)
test.rename(columns = {'id_31':'id-31'}, inplace = True)
test.rename(columns = {'id_32':'id-32'}, inplace = True)
test.rename(columns = {'id_33':'id-33'}, inplace = True)
test.rename(columns = {'id_34':'id-34'}, inplace = True)
test.rename(columns = {'id_35':'id-35'}, inplace = True)
test.rename(columns = {'id_36':'id-36'}, inplace = True)
test.rename(columns = {'id_37':'id-37'}, inplace = True)
test.rename(columns = {'id_38':'id-38'}, inplace = True)


In [7]:
(train.isFraud.sum()/train.isFraud.count())*100

3.4990009144173126

In [8]:
train.dropna(axis=0, subset=['isFraud'], inplace=True)
y = train.isFraud
train.drop(['isFraud'], axis=1, inplace=True)

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [10]:
train.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [13]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [31]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]
many_null_cols

['dist2',
 'D7',
 'id-07',
 'id-08',
 'id-18',
 'id-21',
 'id-22',
 'id-23',
 'id-24',
 'id-25',
 'id-26',
 'id-27']

In [32]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols

['dist2',
 'C3',
 'D7',
 'V98',
 'V101',
 'V102',
 'V103',
 'V104',
 'V105',
 'V106',
 'V107',
 'V108',
 'V109',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V115',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V124',
 'V125',
 'V129',
 'V132',
 'V133',
 'V134',
 'V135',
 'V136',
 'V137',
 'V281',
 'V284',
 'V286',
 'V290',
 'V293',
 'V295',
 'V296',
 'V297',
 'V298',
 'V299',
 'V300',
 'V301',
 'V305',
 'V309',
 'V311',
 'V316',
 'V318',
 'V319',
 'V320',
 'V321',
 'id-07',
 'id-08',
 'id-18',
 'id-21',
 'id-22',
 'id-23',
 'id-24',
 'id-25',
 'id-26',
 'id-27']

In [33]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test))
len(cols_to_drop)

82

In [40]:
#train.rename(columns = {'id_07':'id-07'}, inplace = True)
#train.rename(columns = {'id_08':'id-08'}, inplace = True)

reduced_train = train.drop(cols_to_drop, axis=1)
reduced_test = test.drop(cols_to_drop, axis=1)

In [51]:
#cat_cols = ['id-12', 'id-13', 'id-14', 'id-15', 'id-16', 'id-17', 'id-19', 'id-20', 'id-26', 'id-27', 'id-28', 'id-29',
#            'id-30', 'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
#            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
#            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']

object_cols = [col for col in reduced_train.columns if reduced_train[col].dtype == "object"]
object_cols

for col in object_cols:
    if col in reduced_train.columns:
        le = LabelEncoder()
        le.fit(list(reduced_train[col].astype(str).values) + list(reduced_test[col].astype(str).values))
        reduced_train[col] = le.transform(list(reduced_train[col].astype(str).values))
        reduced_test[col] = le.transform(list(reduced_test[col].astype(str).values))  

In [61]:
imputed_cols = [col for col in reduced_train.columns if reduced_train[col].dtype != "object"]
imputed_cols

for col in imputed_cols:
    if col in reduced_train.columns:
        si = SimpleImputer()
        si.fit(reduced_train[col])
        reduced_train[col] = si.transform(list(reduced_train[col].astype(str).values))
        reduced_test[col] = si.transform(list(reduced_test[col].astype(str).values))  

ValueError: Expected 2D array, got 1D array instead:
array=[2987000. 2987001. 2987002. ... 3577537. 3577538. 3577539.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
imputed_train = reduced_train.select_dtypes(exclude=['object'])
imputed_test = reduced_train.select_dtypes(exclude=['object'])

imputer = SimpleImputer() 
imputed_X_train = pd.DataFrame(imputer.fit_transform(imputed_train))
imputed_X_test = pd.DataFrame(imputer.transform(imputed_test))

In [59]:
for col in imputed_cols:
    if col in reduced_train.columns:
        reduced_train[col] = imputed_X_train[col]
        reduced_test[col] = imputed_X_test[col]

KeyError: 'TransactionID'

In [11]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be label encoded: ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id-12', 'id-15', 'id-16', 'id-23', 'id-27', 'id-28', 'id-29', 'id-35', 'id-36', 'id-37', 'id-38', 'DeviceType']

Categorical columns that will be dropped from the dataset: ['DeviceInfo', 'id-31', 'id-34', 'id-33', 'id-30']


In [12]:
# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    print(col)
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

M1


TypeError: argument must be a string or number

In [69]:
X = reduced_train.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
#y = reduced_train.sort_values('TransactionDT')['isFraud']
#X_test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
X_test = reduced_test.drop(['TransactionDT', 'TransactionID'], axis=1)

test = test[["TransactionDT", 'TransactionID']]

In [82]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [78]:
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }

n_estimators = 5000
n_jobs = -1
verbose = 500
early_stopping_rounds = 200


In [83]:
model = lgb.LGBMClassifier(n_estimators=n_estimators, n_jobs = n_jobs)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric = 'mae', verbose=verbose, early_stopping_rounds=early_stopping_rounds)     
#y_pred_valid = model.predict_proba(X_valid)[:, 1]
#y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]

Training until validation scores don't improve for 200 rounds
[500]	training's l1: 0.0271008	training's binary_logloss: 0.0452529	valid_1's l1: 0.0306418	valid_1's binary_logloss: 0.0604948
[1000]	training's l1: 0.0202667	training's binary_logloss: 0.0312324	valid_1's l1: 0.0261948	valid_1's binary_logloss: 0.0539835
[1500]	training's l1: 0.0155865	training's binary_logloss: 0.0225651	valid_1's l1: 0.0232228	valid_1's binary_logloss: 0.0509376
[2000]	training's l1: 0.0122271	training's binary_logloss: 0.0169329	valid_1's l1: 0.0210381	valid_1's binary_logloss: 0.0492781
[2500]	training's l1: 0.00972852	training's binary_logloss: 0.0129702	valid_1's l1: 0.0194666	valid_1's binary_logloss: 0.0487463
Early stopping, best iteration is:
[2640]	training's l1: 0.00911431	training's binary_logloss: 0.0120326	valid_1's l1: 0.019052	valid_1's binary_logloss: 0.0486701


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [84]:
y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]

In [100]:
output = pd.DataFrame({'TransactionID': test['TransactionID'],
                       'IsFraud': y_pred})
output.to_csv('submission.csv', index=False)

In [101]:
output

,TransactionID,IsFraud
0,3663549,0.000107
1,3663550,0.000018
2,3663551,0.000035
3,3663552,0.000186
4,3663553,0.000193
...,...,...
506686,4170235,0.009119
506687,4170236,0.001154
506688,4170237,0.003253
506689,4170238,0.004456
